## Building up Hamiltonian

Hamiltonian is $H=-t_3 \sum_{layer, s}(c_{c,s}^+c_{d,s}+c_{d,s}^+c_{c,s})-\mu \sum_{layer, s} n_{layer,s} - U (n_{d,+}n_{d,-}) $



In [1]:
using LinearAlgebra

function coordinate(n;L::Int64=2)
    num_sites = L^2
    i::Int64 = Int(ceil(n/L))
    j::Int64 = mod1(n,L)  #site i is at i-th row, j-th column
    return (i,j)
end

function bit_pos(coordinate::Tuple{Int64,Int64};spin_num::Int64=2)
    """
        input (layer#, spin), output position in the bits. 
        layer : 1,2
        spin : 1/2 being up/down
    """
    n = (coordinate[1]-1)*spin_num + coordinate[2]
    return n
end

bit_pos (generic function with 1 method)

In [2]:
bit_pos(coordinate(3))

3

In [3]:
t3 = 1; U = 2
total_layer = 2; spin_num = 2; N = total_layer*spin_num
μ = 0; β=2

2

In [4]:
function Hamiltonian(;total_layer = 2, spin_num = 2, t3 = 1, U = 0, μ = 0, β=2)
    H = zeros(Float64,2^N,2^N)
    for a in 0:(2^N-1) #loop over all states
        a_binary = digits!(zeros(Int64, 64), a, base = 2)
        H[a+1,a+1] += -U * (a_binary[3]*a_binary[4])
        for i in 1:N #loop over all sites in a given state
            H[a+1,a+1] += -μ * a_binary[i]
            neib_pos = bit_pos( (mod1(coordinate(i)[1]+1, total_layer), coordinate(i)[2]) )
            if (a_binary[i] == 1) && (a_binary[neib_pos] == 0)
                b = a ⊻ (1<<(i-1))
                b = b ⊻ (1<<(neib_pos-1))
                H[a+1,b+1] += -t3
            end
        end
    end
    return H
end
# println(H)
vals,vecs=eigen(Hamiltonian(μ=-1))

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
16-element Array{Float64,1}:
 0.0                   
 0.0                   
 1.1102230246251565e-15
 3.552713678800501e-15 
 2.0                   
 2.0                   
 2.0                   
 2.0                   
 2.0                   
 2.0000000000000004    
 2.000000000000001     
 2.0000000000000013    
 3.999999999999999     
 4.0                   
 4.0                   
 4.0                   
eigenvectors:
16×16 Array{Float64,2}:
 1.0   0.0          0.0       0.0  …   0.0           0.0   0.0       0.0
 0.0   0.0          0.707107  0.0      0.0           0.0   0.0       0.0
 0.0   0.707107     0.0       0.0      0.0           0.0   0.0       0.0
 0.0   0.0          0.0       0.5      0.0           0.5   0.0       0.0
 0.0   0.0          0.707107  0.0      0.0           0.0   0.0       0.0
 0.0   0.0          0.0       0.0  …   0.0           0.0   0.0       0.0
 0.0  -3.33067e-16  0.0       0.5      4.

### Now takes measurement of n

In [5]:
function n_measure(;eig_vecs,eig_vals, N)
    z = 0
    for i in 1:2^N
        E = vals[i]
        z += exp(-β*E)
    end
    n = 0
    for i in 1:2^N
        eig_vec = eig_vecs[:,i]
        E = eig_vals[i]
        for basis_state in 0:(2^N-1) #loop over all states
            basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
            n += conj(eig_vec[basis_state+1])*eig_vec[basis_state+1]*sum(basis_state_binary[1:4])*exp(-β*E)
        end
    end
    n = (n / z) / N # n per spin per site
    return n
end

n_measure (generic function with 1 method)

### Now calculate n for different $\mu$ values

In [6]:
n = []
for μ in range(-1,1,length=11)
    total_layer = 2; spin_num = 2; N = total_layer*spin_num
    H = Hamiltonian(μ=μ)
    eig_vals, eig_vecs=eigen(H)
    push!(n, n_measure(;eig_vecs=eig_vecs,eig_vals=eig_vals, N=N))
end
println(n)

Any[0.25899310498104505, 0.49393813929926195, 0.9905225556361604, 2.0907979966516206, 4.651128998195981, 10.935021788312117, 27.30681344939203, 72.92522182360244, 209.78019181486343, 653.570221732219, 2208.910422159742]


In [3]:
L=2
function neib(n::Int64;L::Int64=2)
    coord = coordinate(n,L=L)
    neibs = Tuple{Int64,Int64}[]
    push!(neibs, (mod1(coord[1]+1,L), coord[2]))
    push!(neibs, (mod1(coord[1]-1,L), coord[2]))
    push!(neibs, (coord[1], mod1(coord[2]+1,L)))
    push!(neibs, (coord[1], mod1(coord[2]-1,L)))
    push!(neibs, (mod1(coord[1]+1,L), mod1(coord[2]-1,L)))
    push!(neibs, (mod1(coord[1]-1,L), mod1(coord[1]+1,L)))
    push!(neibs, (mod1(coord[1]+1,L), mod1(coord[2]+1,L)))
    push!(neibs, (mod1(coord[1]-1,L), mod1(coord[2]-1,L)))
    #=convert coordinations to positions in bits=#
    neibs_bit_pos = Set{Int64}()
    for neib in neibs
        push!(neibs_bit_pos, bit_pos(neib,L=L))
    end
    return neibs_bit_pos
end

neib_list = Set{Int64}[]
for n in 1:L^2
    push!(neib_list, neib(n, L=L))
end
println(neib_list)


Set{Int64}[Set([4, 2, 3]), Set([4, 3, 1]), Set([4, 2, 1]), Set([2, 3, 1])]


In [4]:
neib(1)

Set([4, 2, 3])

## Writing the function to flip two spins at position i and j

## Building the Hamiltonian for Heisenberg Model
 $H=J\sum_{i=1}^{N}(S_i^z S_{i+1}^z + 1/2(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)) $


In [5]:
H = zeros(Float64,2^(L^2),2^(L^2))
for a in 0:(2^(L^2)-1) #loop over all states
    a_binary = digits!(zeros(Int64, 64), a, base = 2)
    for i in 1:L^2 #loop over all sites in a given state
        for j in neib_list[i] #loop over(compare) all neighbors of a given site
            if a_binary[i]==a_binary[j]
                H[a+1,a+1] += 1/4/2
            else
                H[a+1,a+1] -= 1/4/2
                b = a ⊻ (1<<(i-1))
                b = b ⊻ (1<<(j-1))
                H[a+1,b+1] = 1/2
            end   
        end
    end
end
println(H)
eigen(H)

[1.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.5 0.0 0.0 0.5 0.0 0.0 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.5 0.0 0.5 0.5 0.0 0.0 0.5 0.5 0.0 0.0 0.0 0.0 0.0; 0.0 0.5 0.5 0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.5 0.0 -0.5 0.5 0.0 0.0 0.5 0.0 0.0 0.5 0.0 0.0 0.0; 0.0 0.0 0.0 0.5 0.0 0.5 -0.5 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.5 0.5 0.0; 0.0 0.5 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0 0.0 -0.5 0.5 0.0 0.5 0.0 0.0 0.0; 0.0 0.0 0.0 0.5 0.0 0.0 0.5 0.0 0.0 0.5 -0.5 0.0 0.5 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.0 0.0 0.0 0.0 0.5 0.5 0.0; 0.0 0.0 0.0 0.0 0.0 0.5 0.5 0.0 0.0 0.5 0.5 0.0 -0.5 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.0 0.0 0.5 0.0 0.0 0.5 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.5 0.0 0.0 0.0 0.5 0.0 0.5 0.0 0.0; 0.0 0.0 0.0 0.0 0.

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
16-element Array{Float64,1}:
 -1.5                
 -1.4999999999999973 
 -0.5                
 -0.5                
 -0.5                
 -0.4999999999999999 
 -0.4999999999999999 
 -0.4999999999999998 
 -0.49999999999999917
 -0.4999999999999982 
 -0.4999999999999982 
  1.5                
  1.5                
  1.5                
  1.5                
  1.5000000000000004 
eigenvectors:
16×16 Array{Float64,2}:
  0.0           0.0           0.0          …  1.0   0.0  0.0   0.0        
 -5.56657e-17   0.0          -0.0612173       0.0   0.0  0.0   0.5        
 -5.56657e-17   0.0          -0.0612173       0.0   0.0  0.0   0.5        
 -1.11022e-16  -0.57735       5.55112e-17     0.0   0.0  0.0  -6.16298e-33
 -5.56657e-17   0.0           0.132223        0.0   0.0  0.0   0.5        
 -0.5           0.288675     -0.696328     …  0.0   0.0  0.0   1.61908e-17
  0.5           0.288675      0.0417741       0.0   0.0  0.0 

# Now Let's Construct Hamiltonian for our problem, by changing neighbor lists


The list of neighbors would be different, because of the lattice configuration.

In [6]:
L=4
function neib(n::Int64;L::Int64=2)
    coord = coordinate(n,L=L)
    neibs = Tuple{Int64,Int64}[]
    push!(neibs, (mod1(coord[1]+1,L), coord[2]))
    push!(neibs, (mod1(coord[1]-1,L), coord[2]))
    push!(neibs, (coord[1], mod1(coord[2]+1,L)))
    push!(neibs, (coord[1], mod1(coord[2]-1,L)))    
    if iseven(coord[1]+coord[2])
        push!(neibs, (mod1(coord[1]+1,L), mod1(coord[2]-1,L)))
        push!(neibs, (mod1(coord[1]-1,L), mod1(coord[2]+1,L)))
    else
        push!(neibs, (mod1(coord[1]+1,L), mod1(coord[2]+1,L)))
        push!(neibs, (mod1(coord[1]-1,L), mod1(coord[2]-1,L)))
    end
    #=convert coordinations to positions in bits=#
    neibs_bit_pos = Set{Int64}()
    for neib in neibs
        push!(neibs_bit_pos, bit_pos(neib,L=L))
    end
    return neibs_bit_pos
end

neib_list = Set{Int64}[]
for n in 1:L^2
    push!(neib_list, neib(n, L=L))
end
println(neib_list)

Set{Int64}[Set([4, 13, 14, 2, 5, 8]), Set([7, 14, 13, 3, 6, 1]), Set([7, 4, 2, 16, 15, 6]), Set([3, 16, 8, 5, 15, 1]), Set([9, 4, 10, 8, 6, 1]), Set([7, 9, 10, 2, 3, 5]), Set([2, 3, 11, 8, 6, 12]), Set([7, 4, 11, 5, 12, 1]), Set([13, 10, 16, 5, 12, 6]), Set([9, 14, 11, 5, 15, 6]), Set([7, 14, 10, 8, 15, 12]), Set([7, 9, 13, 16, 11, 8]), Set([9, 14, 2, 16, 12, 1]), Set([13, 10, 2, 11, 15, 1]), Set([4, 14, 10, 3, 16, 11]), Set([4, 13, 9, 3, 15, 12])]


## Now construct Hamlitonian for our system

Our Hamiltonian is $H=J\sum_{i=1}^{N}\sum_{\sigma}(S_i^z S_{i+\sigma}^z + h S_i^x) $

equivalently, $H=J\sum_{i=1}^{N}\sum_{\sigma}(S_i^z S_{i+\sigma}^z+\frac{h}{2}(S_i^+ + S_i^-))$


In [7]:
J=1
h=1
using SparseArrays
H = spzeros(2^(L^2),2^(L^2))
for a in 0:(2^(L^2)-1) #loop over all states
    a_binary = digits!(zeros(Int64, 64), a, base = 2)
    for i in 1:L^2 #loop over all sites in a given state
        b = a ⊻ (1<<(i-1))
        H[a+1,b+1] = 1/2h
        for j in neib_list[i] #loop over(compare) all neighbors of a given site
            if a_binary[i]==a_binary[j]
                H[a+1,a+1] += 1/4J/2
            else
                H[a+1,a+1] -= 1/4J/2
            end   
        end
    end
end
println(H[1,1])

12.0


## Now Let's construct Parity States

We can do a little rotation of our system and construct Hamiltonian in that "direction". The Hamiltonian should be:
$H=J\sum_{i=1}^{N}\sum_{\sigma}(S_i^x S_{i+\sigma}^x - h S_i^z)$

Now we have a nice Parity symmetry comes in handy. The Operator is $\prod_i \sigma_i^z$. The $\sigma_i^z$ basis is exactly the eigenbasis of the parity operator. The parity operator commutes with H, thus divides the Hamiltonian into 2 block diagonal parts. With one part having even total spin, one part having odd total spin. 

Physically we can see this by making use of the fact that $S_i^x S_{i+\sigma}^x$ flips 2 spins at a time. So different sectors can't mix.

First we need to loop over all states, and construct a list of even and odd states.

In [8]:
function chk_parity(state::Int64)
    state_binary = digits!(zeros(Int64, 64), state, base = 2)
    if iseven(sum(state_binary))
        return :even
    else
        return :odd
    end
end

function parity_states_list(;L::Int64=L)
    even_state = Int64[]
    odd_state = Int64[]
    even_state_num = Dict{Int64, Int64}()
    odd_state_num = Dict{Int64, Int64}()
    even_state_tot = 0
    odd_state_tot = 0
    for state in 0:(2^(L^2)-1)
        if chk_parity(state) == :even
            even_state_tot += 1
            push!(even_state, state)
            even_state_num[state] = even_state_tot
        else
            odd_state_tot += 1
            push!(odd_state, state)
            odd_state_num[state] = odd_state_tot
        end
    end
    return Dict{Symbol, Any}(:even_state => even_state, :odd_state => odd_state, :even_state_num => even_state_num, :odd_state_num => odd_state_num, :even_state_tot => even_state_tot, :odd_state_tot => odd_state_tot)
end

L=2
result = parity_states_list(L=L)

Dict{Symbol,Any} with 6 entries:
  :even_state     => [0, 3, 5, 6, 9, 10, 12, 15]
  :even_state_num => Dict(0=>1,9=>5,10=>6,3=>2,5=>3,15=>8,6=>4,12=>7)
  :even_state_tot => 8
  :odd_state_tot  => 8
  :odd_state_num  => Dict(7=>4,4=>3,13=>7,14=>8,2=>2,11=>6,8=>5,1=>1)
  :odd_state      => [1, 2, 4, 7, 8, 11, 13, 14]

## Now we could construct Hamiltonian for even and odd sectors

In [9]:
using SparseArrays
using Arpack
L=2
state_gen = parity_states_list(L=L)
even_state = state_gen[:even_state]
even_state_num = state_gen[:even_state_num]
odd_state_num  = state_gen[:odd_state_num]
odd_state = state_gen[:odd_state]
even_state_tot = state_gen[:even_state_tot]
odd_state_tot = state_gen[:odd_state_tot]
# even sector
function H_even(;L::Int64=L, J=1, h=1)
    H = zeros(even_state_tot,even_state_tot)
    for state in even_state #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:L^2 #loop over all sites i in a given state
            #diagonal terms interact with h
            if state_binary[i] == 1
                H[even_state_num[state],even_state_num[state]] += h/2
            else
                H[even_state_num[state],even_state_num[state]] -= h/2
            end   
            # off diagonal terms come from flipping bonds, j is neighbor of i
            for j in neib_list[i] 
                fliped_state = state ⊻ (1<<(i-1))
                fliped_state = fliped_state ⊻ (1<<(j-1))
                H[even_state_num[state],even_state_num[fliped_state]] = J
            end
        end
    end
    return H
end
println(H_even())
#eigen(H_even())

#odd sector
function H_odd(;L::Int64=2, J=1, h=1, neib_list, odd_state, odd_state_num, odd_state_tot)
    H = spzeros(odd_state_tot,odd_state_tot)
    for state in odd_state #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:L^2 #loop over all sites i in a given state
            #diagonal terms interact with h
            if state_binary[i] == 1
                H[odd_state_num[state],odd_state_num[state]] += h/2
            else
                H[odd_state_num[state],odd_state_num[state]] -= h/2
            end   
            # off diagonal terms come from flipping bonds, j is neighbor of i
            for j in neib_list[i] 
                fliped_state = state ⊻ (1<<(i-1))
                fliped_state = fliped_state ⊻ (1<<(j-1))
                H[odd_state_num[state],odd_state_num[fliped_state]] = J/4
            end
        end
    end
    return H
end



KeyError: [91mKeyError: key 4097 not found[39m

# Now Finally! Let's Construct Momentum States.

## First we should find all reference states: Constructing a list of reference states

### In order to construct a list of reference states, we first need to define the symmetry of the system, as well as the symmetry operation on the system

In [10]:
function pos_translation(pos::Int64, dist::Int64, direction::Symbol; L::Int64)
    if direction == :x
         
    elseif direction == :y
        
    else
        println("Direction not defined.")
    end
end

function translation(state::Int64; direction::Symbol, dist::Int64, L::Int64)
    max_len = L^2
    state_binary = digits!(zeros(Int64, 64), state, base = 2)
    translated_state = 0
    if direction == :x
        for pos in 1:max_len
            if state_binary[pos] == 1
                #translated_pos = mod1(pos + dist, L) + Int(floor((pos-0.5)/L))*L
                #translated_state = translated_state ⊻ (1<<(translated_pos-1))
                pos_coord = coordinate(pos,L=L)
                trans_pos_coord_i = pos_coord[1]
                trans_pos_coord_j = mod1(pos_coord[2] + dist, L)
                trans_pos = bit_pos((trans_pos_coord_i, trans_pos_coord_j), L=L)
                translated_state = translated_state ⊻ (1<<(trans_pos-1))
            end
        end
        return translated_state
    elseif direction == :y
        for pos in 1:max_len
            if state_binary[pos] == 1
                #translated_pos = Int((mod1(ceil((pos-0.5)/L) + dist, L)-1)*L + mod1(pos, L))
                #translated_state = translated_state ⊻ (1<<(translated_pos-1))
                pos_coord = coordinate(pos,L=L)
                trans_pos_coord_i = mod1(pos_coord[1] + dist, L)
                trans_pos_coord_j = pos_coord[2]
                trans_pos = bit_pos((trans_pos_coord_i, trans_pos_coord_j), L=L)
                translated_state = translated_state ⊻ (1<<(trans_pos-1))
            end
        end
        return translated_state
    else
        error("Direction not defined.")
    end
end
translation(9, direction=:x, dist=-1; L=2)

6

check routine to check whether a state s belong to a certain k subspace, as well as determining the periodicity of the state under translation of 2 different directions.

In [11]:
function checkstate(s::Int64; k::Real, direction::Symbol, L::Int64)
    R = -1; t = s;
    for i in 1:L
        t = translation(t, direction=direction, dist=1; L=L)
        if t < s
            #println("t<s")
            return (false, R)
        elseif t == s
            R = i
            if mod(k, L/i) != 0
                #println("$(L/i)")
                return (false, R)
            else
                #println("should be true")
                return (true, R)
            end
        end
    end
end

# checkstate for a 2D system, with k being a 2D Tuple (kx, ky), Turned out to be wrong
function checkstate(s::Int64; k::Tuple, L::Int64)
    Rx = -1; Ry = -1; t = s;
    for i in 1:L
        t = translation(t, direction=:x, dist=1; L=L)
        for j in 1:L
            t = translation(t, direction=:y, dist=1; L=L)
            # Check if the state is the smallest integer and return periodicity Rx, Ry
            if t < s
                #println("t<s")
                return (false, (Rx, Ry))
            elseif t == s
                Rx = i; Ry = j
                if (mod(k[1], L/i) != 0) || (mod(k[2], L/j) != 0)
                    println("$(L/i)")
                    return (false, (Rx, Ry))
                else
                    #println("should be true")
                    return (true, (Rx, Ry))
                end
            end
            #finished check states for one certain translation
        end
    end
end

#Check whether a state is ref state 2D, if it's a ref state, return number of different states Da
function checkstate(s::Int64; k::Tuple, L::Int64, ϵ::Float64 = 10^(-10))
    k = 2pi/L .* k
    states = Set{Int64}() 
    push!(states, s)
    is_ref = true
    Fa = 0
    t = s
    for i in 1:L
        t = translation(t, direction=:x, dist=1; L=L)
        for j in 1:L
            t = translation(t, direction=:y, dist=1; L=L)
            push!(states, t)
            # Check if the state is the smallest integer and return periodicity Rx, Ry
            if t < s
                is_ref = false                
            end
            if t == s
                Fa += exp(-im*(k[1]*i + k[2]*j))
            end
            #finished check states for one certain translation
        end
    end
    if abs(Fa) < ϵ
       is_ref = false
    end
    return (is_ref, length(states), real(Fa))
end

# checkstate for a 2D system, with k being a 2D Tuple (kx, ky)
function ref_state_gen(;k::Tuple, L::Int64)
    ref_state = Int64[]#Set{Int64}()
    Da = Int64[]
    Fa = []
    for state in 0:(2^(L^2)-1)
        chk_state = checkstate(state, k=k, L=L)
        if chk_state[1]
            push!(ref_state, state)
            push!(Da, chk_state[2])
            push!(Fa, chk_state[3])
        end
    end
    return (ref_state, Da, Fa)
end
-
function k_check(state::Int64;L::Int64)
    s
end
ref_state_gen(k=(2,2),L=4)

([1, 3, 5, 7, 17, 18, 19, 20, 21, 22  …  32191, 32251, 32255, 32475, 32479, 32511, 32639, 32703, 32735, 32767], [16, 16, 8, 16, 16, 16, 16, 16, 16, 16  …  16, 16, 16, 4, 16, 16, 8, 16, 8, 16], Any[1.0 + 9.797174393178826e-16im, 1.0 + 9.797174393178826e-16im, 2.0 + 1.7145055188062944e-15im, 1.0 + 9.797174393178826e-16im, 1.0 + 9.797174393178826e-16im, 1.0 + 9.797174393178826e-16im, 1.0 + 9.797174393178826e-16im, 1.0 + 9.797174393178826e-16im, 1.0 + 9.797174393178826e-16im, 1.0 + 9.797174393178826e-16im  …  1.0 + 9.797174393178826e-16im, 1.0 + 9.797174393178826e-16im, 1.0 + 9.797174393178826e-16im, 4.0 + 2.449293598294706e-15im, 1.0 + 9.797174393178826e-16im, 1.0 + 9.797174393178826e-16im, 2.0 + 1.7145055188062944e-15im, 1.0 + 9.797174393178826e-16im, 2.0 + 1.4695761589768238e-15im, 1.0 + 9.797174393178826e-16im])

In [13]:
real(1+im)

1

### Now we can construct a few list of reference states for different ks. Along with its periodicity. 

We represent a k value by an integer $n = {1, 2, ..., L_T}$ multiplying by $\frac{2\pi}{R}$ with $R$ being the periodicity of the state. For $R\ne L_T$, allowed k values are $\frac{2\pi}{N} \frac{N}{R} n$.So we have to choose integer from the list that is a multiple of $\frac{N}{R}$.

For a lattice that is more complicated, $L_T$ should equal to the period of the lattice on x or y direction, not neccesarily equal to the number of sites on each direction $L$. For example the square ice ising model lattice would have L equal to half the number of sites each direction, because the lattice repeats itself only twice the sites.

In [19]:
# first settle the periodicity of the lattice Lt. For a simple square lattice, Lt = L
L = 2
Lt = L
# using this we could construct lists of momentum states
function momentum_state_list_gen(;Lt::Int64 , L::Int64)
    ref_state_tot = Array{Int64}(undef, Lt, Lt)
    ref_state = Array{Array{Int64,1}}(undef, Lt, Lt)
    ref_state_period = Array{Array{Tuple{Int64, Int64},1}}(undef, Lt, Lt)
    ref_state_num = Array{Dict{Int64, Int64}}(undef, Lt, Lt)
    for kx in 1:Lt
        for ky in 1:Lt
            ref_state_tot[kx, ky] = 0
            ref_state[kx, ky] = Int64[]
            ref_state_period[kx, ky] = Tuple{Int64, Int64}[]
            ref_state_num[kx, ky] = Dict{Int64, Int64}()
            for state in 0:(2^(L^2)-1)
                chk_state = checkstate(state; k=(kx, ky), L=L)
                if chk_state[1]
                    ref_state_tot[kx, ky] += 1
                    push!(ref_state[kx, ky], state)
                    ref_state_num[kx, ky][state] = ref_state_tot[kx, ky]
                    push!(ref_state_period[kx, ky], (chk_state[2][1], chk_state[2][2]))
                end
            end
        end
    end
    return Dict{Symbol, Any}(:ref_state_tot => ref_state_tot, :ref_state => ref_state, :ref_state_period => ref_state_period, :ref_state_num => ref_state_num)
end

momentum_state_list_gen (generic function with 1 method)

### For each state, build a function to find how it can be translated to a momentum state

In [18]:
# return the number of translation in x and y direction in a Tuple (i, j)
function find_ref(s::Int64; L::Int64)
    t = s
    ref = t
    trans = (0,0)
    for i in 1:L
        t = translation(t, direction=:x, dist=1; L=L)
        for j in 1:L
            t = translation(t, direction=:y, dist=1; L=L)
            if t < ref
                ref = t
                trans = (i, j)  
            end
        end
    end
    return Dict(:trans => trans, :ref => ref)
end
find_ref(9,L=2)

Dict{Symbol,Any} with 2 entries:
  :trans => (1, 2)
  :ref   => 6

### Now construct Hamiltonian for each momentum sector using the previously generated momentum states

The Hamiltonian is:
$H=\sum_{i=1}^{N}\sum_{\sigma}(\frac{J}{4}(S_i^+ + S_i^-)(S_{i+\sigma}^+ + S_{i+\sigma}^-) +h S_i^z)$


We know that the K-space hamiltonian for the j-th bond among all the bonds is :

$<b_j(\vec{k})|H_j|a(\vec{k})> = h_j(a) e^{-\vec{l_j} \cdot \vec{k}} \sqrt{\frac{N_{b_j}}{N_a}}$

So the Hamiltonian corresponding to the $|a(\vec{k})>$ column should be the sum of all the above terms over $j$.

In [ ]:
@time momentum_state_list = momentum_state_list_gen(;Lt = Lt, L = L)[:ref_state]
state_gen = parity_states_list(L=L)
even_state = state_gen[:even_state]
even_state_num = state_gen[:even_state_num]
odd_state_num  = state_gen[:odd_state_num]
odd_state = state_gen[:odd_state]
even_state_tot = state_gen[:even_state_tot]
odd_state_tot = state_gen[:odd_state_tot]
# even sector
function H_even(;L::Int64=L, J=1, h=1)
    H = zeros(even_state_tot,even_state_tot)
    for state in even_state #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:L^2 #loop over all sites i in a given state
            #diagonal terms interact with h
            if state_binary[i] == 1
                H[even_state_num[state],even_state_num[state]] += h/2
            else
                H[even_state_num[state],even_state_num[state]] -= h/2
            end   
            # off diagonal terms come from flipping bonds, j is neighbor of i
            for j in neib_list[i] 
                fliped_state = state ⊻ (1<<(i-1))
                fliped_state = fliped_state ⊻ (1<<(j-1))
                H[even_state_num[state],even_state_num[fliped_state]] = J
            end
        end
    end
    return H
end
println(H_even())

In [107]:
m = Array{Dict{Int64, Int64}}(undef, 2, 2)
m[1,1] = Dict{Int64, Int64}()
m[1,1][1] = 2
println(m[1,1][1])

2


In [41]:
zeros(Float64,2,2)

2×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0

In [20]:
bitstring(1 ⊻ (1<<1))

"0000000000000000000000000000000000000000000000000000000000000011"

In [62]:
digits!(zeros(Int64, 64), 1, base = 2)

64-element Array{Int64,1}:
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [8]:
using SparseArrays

In [13]:
L=4
A = spzeros(2^(L^2),2^(L^2))
println(A)

65536×65536 SparseMatrixCSC{Float64,Int64} with 0 stored entries


In [11]:
A[1,2] =2.0

2.0

In [12]:
println(A)


  [1, 1]  =  1.0
  [1, 2]  =  2.0
  [2, 2]  =  0.0
  [3, 3]  =  1.0


In [24]:
Pkg.add("Arpack")

UndefVarError: [91mUndefVarError: Pkg not defined[39m

In [1]:
L=2
zeros(Float64, 2^(L^2))

16-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0